# Cohere reranker

>[Cohere](https://cohere.ai/about)는 기업이 인간-기계 상호작용을 개선할 수 있도록 돕는 자연어 처리 모델을 제공하는 캐나다의 스타트업입니다.

이 노트북은 `retriever`에서 [Cohere의 rerank endpoint](https://docs.cohere.com/docs/reranking)를 사용하는 방법을 보여줍니다. 

In [ ]:
# 설치
# !pip install -qU cohere

## Cohere API 키 설정

- [API 키 발급](https://dashboard.cohere.com/api-keys)
- `.env` 파일에 `COHERE_API_KEY` 키 값에 API 키를 넣어주세요.

**참고**
- [공식 도큐먼트](https://docs.cohere.com/docs/the-cohere-platform?_gl=1*st323v*_gcl_au*MTA2ODUyNDMyNy4xNzE4MDMzMjY2*_ga*NTYzNTI5NDYyLjE3MTgwMzMyNjc.*_ga_CRGS116RZS*MTcyMTk4NzMxMi4xMS4xLjE3MjE5ODczNjIuMTAuMC4w)
- [Reranker Model 리스트](https://docs.cohere.com/docs/rerank-2)

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
from langchain_chroma import Chroma

# API KEY 정보로드
load_dotenv()

## 사용법

In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

**Cohere 다국어 지원 모델**

- Embedding: `embed-multilingual-v3.0`, `embed-multilingual-light-v3.0`, `embed-multilingual-v2.0`
- Reranker: `rerank-multilingual-v3.0`, `rerank-multilingual-v2.0`

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로드
documents = TextLoader("./data/2025_사우회선출.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 검색기 초기화
retriever = Chroma.from_documents(
    texts, OpenAIEmbeddings(model="text-embedding-3-small")
).as_retriever(search_kwargs={"k": 10})

# 질의문
query = "사우회장과 같이 활동하는 총무의 후보 기준은 무엇인가요?"

# 문서 검색
docs = retriever.invoke(query)

# 문서 출력
pretty_print_docs(docs)

## CohereRerank을 사용한 재정렬
이제 기본 `retriever`를 `ContextualCompressionRetriever`로 감싸보겠습니다. Cohere 재정렬 엔드포인트를 사용하여 반환된 결과를 재정렬하는 `CohereRerank`를 추가할 것입니다.
CohereRerank에서 모델 이름을 지정하는 것이 필수임을 유의하십시오!

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

# 문서 재정렬 모델 설정
compressor = CohereRerank(model="rerank-multilingual-v3.0")

# 문맥 압축 검색기 설정
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("사우회장과 같이 활동하는 총무의 후보 기준은 무엇인가요?")

# 압축된 문서 출력
pretty_print_docs(compressed_docs)

## 또 다른 예제

"대한민국의 수도는?"라는 질문을 물어보기 위한 복잡한 문서를 모아둔 것이다.

In [ ]:
from langchain.schema import Document

def get_sample_docs():
    return [
        Document(
            page_content="미국의 수도는 워싱턴이고, 일본은 도쿄이며 북한은 평양이다."
        ),
        Document(
            page_content="대한민국은 동아시아에 위치한 나라로, 수도는 부산이라고 잘못 알려진 경우도 있습니다."
        ),
        Document(
            page_content="서울은 대한민국의 수도로, 정치, 경제, 문화의 중심지입니다."
        ),
        Document(
            page_content="많은 사람들이 대구를 대한민국의 수도로 착각하지만, 실제 수도는 아닙니다."
        ),
        Document(page_content="한국은 세계적으로 유명한 도시인 서울이 수도이다."),
        Document(page_content="대한민국의 가장 큰 도시는 인천이지만, 수도는 아닙니다."),
        Document(
            page_content="서울은 대한민국의 수도로서, 1948년부터 공식적으로 지정되었습니다."
        ),
        Document(
            page_content="한국의 수도는 평양이라는 오해가 있을 수 있지만, 이는 북한의 수도입니다."
        ),
        Document(
            page_content="미국의 수도는 워싱턴이고, 재팬은 도교이며 코리아는 서울이다."
        ),
        Document(page_content="대한민국은 동아시아에 위치한 나라이며, 분단국가이다."),
    ]

In [ ]:
from langchain_openai import OpenAIEmbeddings


vector_store = Chroma.from_documents(get_sample_docs(), OpenAIEmbeddings(model="text-embedding-3-small"))
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

In [ ]:
query = "대한민국의 수도는?"

docs = retriever.invoke(query)

pretty_print_docs(docs)

## Reranker를 통해 재정렬

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

# 문서 재정렬 모델 설정
compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=4)

# 문맥 압축 검색기 설정
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("대한민국의 수도는?")

# 압축된 문서 출력
pretty_print_docs(compressed_docs)